In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")

c:\Users\Tridib Roy Chowdhury\OneDrive\Desktop\CODING\PDFQNAChatBot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('Operating Systems Course for Beginners.pdf')
docs=loader.load()


In [3]:
docs

[Document(metadata={'producer': 'Skia/PDF m131', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/131.0.0.0 Safari/537.36', 'creationdate': '2025-11-06T17:14:30+00:00', 'title': 'Operating Systems Course for Beginners', 'moddate': '2025-11-06T17:14:30+00:00', 'source': 'Operating Systems Course for Beginners.pdf', 'total_pages': 239, 'page': 0, 'page_label': '1'}, page_content='📚 Operating Systems Fundamentals Overview\nBrief Overview\nThis note covers Operating System fundamentals and was created from the Operating Systems Course for Beginners YouTubevideo. The material spans approximately 25 hours of instruction, with topics ranging from OS architecture and hardware interaction toprocess management, memory handling, and basic I/O; the course emphasizes fundamental theory, numerical examples, and periodicrevision to reinforce learning. As you dive in, remember the course prerequisites include basic computer architecture knowledge andint

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
split=splitter.split_documents(docs)

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [5]:
from langchain_chroma import Chroma
vectorstore=Chroma.from_documents(documents=split,embedding=embeddings)

In [6]:
retriver=vectorstore.as_retriever()


In [7]:
from langchain_community import chat_message_histories
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

In [8]:
store={}

def get_history(id:str)->BaseChatMessageHistory:
  if id not in store:
    store[id]=ChatMessageHistory()

  return store[id]



In [9]:
response=RunnableWithMessageHistory(llm,get_session_history=get_history)

In [10]:
config={"configurable":{"session_id":"chat1"}}


In [12]:
type(config)

dict

In [11]:
from langchain_core.messages import HumanMessage


In [12]:
answer=response.invoke(
    [
        HumanMessage(content="My name is Tridib . I am an ML Engineer")
        
    ],
    config=config
)
answer.content

'Nice to meet you, Tridib! As a Machine Learning (ML) Engineer, you must be working on some exciting projects. What areas of ML do you specialize in, such as computer vision, natural language processing, or time series forecasting?'

In [15]:

answer=response.invoke(
    [
        HumanMessage(content="What is my name")
        
    ],
    config=config
)
answer.content

"Your name is Tridib. You're an ML Engineer."

In [16]:
def response_from_model(message,message_config:dict)->str:
    answer=response.invoke(
        [HumanMessage(content=message)],
        config=message_config
    )
    return answer.content

In [17]:
response_from_model("What is my favourite book",config)

"I don't have any information about your favorite book. We've just started our conversation, and I don't have any personal details or preferences about you. Would you like to share your favorite book with me?"

In [18]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='My name is Tridib . I am an ML Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Tridib, nice to meet you. As an ML Engineer, I'm sure you must be working on exciting projects involving machine learning and deep learning algorithms. What areas of ML are you most interested in or have you worked on (e.g. Computer Vision, NLP, Time Series Forecasting, etc.)?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 47, 'total_tokens': 113, 'completion_time': 0.104027257, 'prompt_time': 0.002256914, 'queue_time': 0.051912216, 'total_time': 0.106284171}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--38eb6163-b9bf-40ff-8a6e-b7a4a7688e79-0', usage_metadata={'input_tokens': 47, 'output_tokens': 66, 'total_tokens'

In [19]:
response_from_model("I think I am a little pookie",config)

'It sounds like you might be referring to the popular children\'s book "Goodnight Moon" by Margaret Wise Brown, where the main character says "Goodnight, little pookie" before going to sleep. Am I close?'

In [20]:
response_from_model("Tell me about myself. Give them pointwise",config)

'Based on our conversation, here are some points about yourself, Tridib:\n\n1. **Name**: Tridib\n2. **Occupation**: ML Engineer (Machine Learning Engineer)\n3. **Interests**: Not explicitly mentioned, but likely related to machine learning and deep learning\n4. **Favorite book**: Not mentioned (although you mentioned a phrase from "Goodnight Moon")\n5. **Personality traits**: Playful, as evidenced by saying "I think I am a little pookie"\n6. **Conversation topics**: Machine learning, books, and general conversation\n\nPlease note that these points are based on our very limited conversation so far, and there may be many more aspects to your personality and interests that we haven\'t discussed yet!'

In [ ]:
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [15]:
system_prompt=(
    "You are an assistant for question answer"
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer,say thay you"
    "don't know . Use three sentences maximum and keep the"
    "answer concise"
    "\n\n"
    "{context}"
    
)

prompt=ChatPromptTemplate.from_messages(
  [  ("system",system_prompt),
      ("human","{input}")
   
   ]
)


In [16]:
document_chain=create_stuff_documents_chain(llm,prompt)

In [17]:
rag_chain=create_retrieval_chain(retriver,document_chain)

In [18]:
answer=rag_chain.invoke({"input":"What is Operating System"})
answer

{'input': 'What is Operating System',
 'context': [Document(id='53d355fa-66d0-4ec5-ae40-bbeecb29c607', metadata={'source': 'Operating Systems Course for Beginners.pdf', 'producer': 'Skia/PDF m131', 'page': 0, 'total_pages': 239, 'title': 'Operating Systems Course for Beginners', 'creationdate': '2025-11-06T17:14:30+00:00', 'page_label': '1', 'moddate': '2025-11-06T17:14:30+00:00', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/131.0.0.0 Safari/537.36'}, page_content='📚 Operating Systems Fundamentals Overview\nBrief Overview\nThis note covers Operating System fundamentals and was created from the Operating Systems Course for Beginners YouTubevideo. The material spans approximately 25 hours of instruction, with topics ranging from OS architecture and hardware interaction toprocess management, memory handling, and basic I/O; the course emphasizes fundamental theory, numerical examples, and periodicrevision to reinforce learning. As you di

In [19]:
answer['answer']

'An Operating System (OS) is an interface between the user and the computer, managing hardware resources and providing services to applications.'

In [24]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt=(
    "Given a chat history and the latest user question"
    "which might reference context in the chat history"
    "formulate a standalone question which can be understood"
    "without the chat history. Do NOT answer the question"
    "just reformulate it if needed and otherwise return it as is."
)



In [25]:
contextualize_q_prompts=ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [27]:
history_aware_retriever=create_history_aware_retriever(llm,retriver,contextualize_q_prompts)


In [28]:
qa_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [29]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [30]:
conversational_rag_chain=RunnableWithMessageHistory(
    rag_chain,
    get_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [34]:
conversational_rag_chain.invoke(
    {"input":"What was the last question I asked"},
    config={
        'configurable':{"session_id":"abc123"}

    }
)["answer"]

'Your last question was "What Operating System?"'